<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V4/Vers%C3%A3o_4_(compara%C3%A7%C3%A3o_balanceamento_smote_manual).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este estudo propõe um modelo de aprendizado de máquina para prever o risco de metástase hepática e/ou pulmonar em pacientes com câncer colorretal (CRC). A partir da base de dados SEER, foram extraídos dados aproximadamente 53 mil pacientes com diagnóstico patológico de CRC entre 2010 e 2015, desenvolvendo sete modelos de algoritmos(Decision tree, Randon Forest, Naive Bayes,  KNN,XGBoost, Gradient Boosting.

### Parte 1:  Importação das Bibliotecas e Carregamento do Dataset

Nesta etapa, importamos as bibliotecas necessárias para análise e carregamos o dataset. Realizamos uma verificação inicial para identificar e remover valores faltantes e definimos as variáveis preditoras (X) e as variáveis alvo (y), preparando os dados para o pré-processamento e a modelagem.

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, average_precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd


# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/dataset/refs/heads/main/export.csv')
df.dropna(inplace=True)

# Definir as variáveis preditoras e a variável alvo
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis']]

y_liver = df['SEER Combined Mets at DX-liver (2010+)']
y_lung = df['SEER Combined Mets at DX-lung (2010+)']

y = pd.concat([y_liver, y_lung], axis=1)

###Parte 2:  Preparação das Variáveis Alvo e Codificação de Variáveis Categóricas

Nesta etapa, preparamos as variáveis alvo (y), combinando as informações de metástase hepática e pulmonar em uma coluna binária para indicar a presença de metástase. Também aplicamos LabelEncoder para transformar variáveis categóricas de X em valores numéricos, facilitando o uso dos dados em modelos de aprendizado de máquina.

In [18]:
y = pd.concat([y_liver, y_lung], axis=1)

# Aplicar codificação a variáveis categóricas em 'X' usando LabelEncoder, para prepará-las para o modelo
for col in X.columns:
    if X[col].dtype == 'object':  # Verifica se a coluna é categórica (strings)
        X[col] = LabelEncoder().fit_transform(X[col])

# Função para combinar as informações de metástase hepática e pulmonar em uma coluna binária 'Binary Mets'
def combine_mets_binary(row):
    if row['SEER Combined Mets at DX-liver (2010+)'] == 'Yes' or row['SEER Combined Mets at DX-lung (2010+)'] == 'Yes':
        return 1  # Com metástase
    else:
        return 0  # Sem metástase

# Aplicar a função para criar a nova coluna binária 'Binary Mets' em 'y'
y['Binary Mets'] = y.apply(combine_mets_binary, axis=1)

# Verificar se 'X' e 'y' têm o mesmo número de amostras
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Salvar o DataFrame 'y' em um arquivo CSV para referência futura ou análise adicional
y.to_csv('/content/Y.csv')

<ipython-input-18-9761f72aaee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-18-9761f72aaee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-18-9761f72aaee8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Tamanho de X: 53448
Tamanho de y: 53448


###Parte 3: implementação manual do smote

In [22]:
from sklearn.neighbors import NearestNeighbors
# Função para implementar o SMOTE manualmente
def manual_smote(X, y, minority_class, n_neighbors=5, n_samples=None):
    """
    Implementa o SMOTE manualmente para gerar amostras sintéticas.

    Parâmetros:
        X (numpy array): Features dos dados.
        y (numpy array): Rótulos dos dados.
        minority_class: Classe minoritária no dataset.
        n_neighbors (int): Número de vizinhos a considerar para a geração.
        n_samples (int): Número de amostras sintéticas a gerar (opcional).

    Retorna:
        X_resampled, y_resampled: Dados balanceados com amostras sintéticas.
    """
    # Identificar os índices das amostras da classe minoritária
    minority_indices = np.where(y == minority_class)[0]
    X_minority = X[minority_indices]

    # Número de amostras sintéticas a gerar
    if n_samples is None:
        n_samples = len(X) - 2 * len(X_minority)

    # Encontrar os vizinhos mais próximos
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(X_minority)
    neighbors = nbrs.kneighbors(X_minority, return_distance=False)

    # Gerar amostras sintéticas
    synthetic_samples = []
    for _ in range(n_samples):
        idx = np.random.choice(len(X_minority))
        neighbor_idx = np.random.choice(neighbors[idx, 1:])  # Escolher um vizinho aleatoriamente

        point_1 = X_minority[idx]
        point_2 = X_minority[neighbor_idx]

        # Criar um ponto sintético
        synthetic_sample = point_1 + np.random.rand() * (point_2 - point_1)
        synthetic_samples.append(synthetic_sample)

    # Adicionar as amostras sintéticas aos dados originais
    X_synthetic = np.array(synthetic_samples)
    y_synthetic = np.full(len(synthetic_samples), minority_class)

    X_resampled = np.vstack([X, X_synthetic])
    y_resampled = np.hstack([y, y_synthetic])

    return X_resampled, y_resampled

# Wrapper para chamar a função manual de SMOTE
def smote_wrapper(X, y, minority_class=1, n_neighbors=5, n_samples=500):
    return manual_smote(X, y, minority_class=minority_class, n_neighbors=n_neighbors, n_samples=n_samples)

# Lista de técnicas de balanceamento
smote_techniques = {
    "SMOTE": smote_wrapper,
}

###Parte 4: Definição e Configuração dos Modelos de Aprendizado de Máquina e Validação Cruzada

Aqui, configuramos os principais algoritmos de aprendizado de máquina, incluindo Decision Tree, Random Forest, SVM, Naive Bayes, KNN, XGBoost e Gradient Boosting. Cada modelo é definido com parâmetros específicos para otimizar o desempenho. Em seguida, aplicamos uma validação cruzada estratificada com 5 divisões para avaliar e comparar a performance dos modelos de maneira consistente e robusta.

In [23]:
# Definição dos modelos de aprendizado de máquina com hiperparâmetros ajustados
models = {
    "Decision Tree": DecisionTreeClassifier(
        criterion='gini',
        max_depth=5,
        min_samples_leaf=10,
        min_samples_split=2,
        random_state=42
    ),
    "Random Forest": RandomForestClassifier(
        bootstrap=True,
        criterion='entropy',
        max_depth=15,
        min_samples_leaf=5,
        min_samples_split=2,
        n_estimators=300,
        random_state=42
    ),
    "SVM": SVC(
        kernel='poly',
        gamma='scale',
        degree=3,
        C=10,
        probability=True,
        random_state=42
    ),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(
        leaf_size=20,
        metric='manhattan',
        n_neighbors=11,
        weights='uniform'
    ),
    "XGBoost": XGBClassifier(
        colsample_bytree=0.6,
        learning_rate=0.1,
        max_depth=6,
        n_estimators=100,
        reg_alpha=0.1,
        reg_lambda=10.0,
        subsample=1.0,
        random_state=42
    ),
    "Gradient Boosting": GradientBoostingClassifier(
        max_depth=3,
        n_estimators=200,
        learning_rate=0.1,
        subsample=0.8,
        min_samples_leaf=5,
        min_samples_split=2,
        random_state=42
    )
}




# Configuração da validação cruzada estratificada com 5 divisões (folds)
# Isso garante que a proporção de classes seja mantida em cada divisão, e o shuffle embaralha os dados antes de dividir
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Parte 5: Avaliação e Comparação dos Modelos de Aprendizado de Máquina em Conjuntos de Treino, Validação e Teste


Este bloco de código implementa a validação cruzada para treinar e avaliar os modelos de aprendizado de máquina definidos no pipeline. Ele utiliza a técnica de K-Fold Cross-Validation para dividir os dados em múltiplos folds, garantindo uma avaliação robusta do desempenho dos modelos. Durante cada fold, os dados de treinamento são balanceados utilizando o SMOTE e escalados com o StandardScaler. Métricas de desempenho, como precisão, recall, F1-Score, especificidade, AUC-ROC e AUPR, são calculadas tanto para o conjunto de treinamento quanto para o conjunto de teste. Além disso, visualizações como matrizes de confusão e curvas ROC e Precisão-Recall são geradas. Ao final, as métricas médias de todos os folds são compiladas para comparação.

In [24]:
results = []
# Iteração entre técnicas de balanceamento e modelos
for smote_name, smote in smote_techniques.items():
    for model_name, model in models.items():
        print(f"\nAplicando {smote_name} com {model_name}")
        fold_metrics = []

        for train_index, test_index in kf.split(X, y['Binary Mets']):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y['Binary Mets'].iloc[train_index], y['Binary Mets'].iloc[test_index]

            # Converter para arrays NumPy
            X_train_np = X_train.values if hasattr(X_train, 'values') else X_train
            y_train_np = y_train.values if hasattr(y_train, 'values') else y_train

            # Aplicar o SMOTE manual
            X_train_res, y_train_res = smote(X_train_np, y_train_np, minority_class=1, n_neighbors=10, n_samples=500)

            # Normalizar os dados
            scaler = StandardScaler()
            X_train_res = scaler.fit_transform(X_train_res)
            X_test = scaler.transform(X_test)

            # Treinar o modelo
            model.fit(X_train_res, y_train_res)
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)[:, 1]

            # Avaliar as métricas
            f1 = f1_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred_proba)
            auc_pr = average_precision_score(y_test, y_pred_proba)
            fold_metrics.append((f1, auc, auc_pr))

        # Métricas médias
        avg_f1, avg_auc, avg_auc_pr = np.mean(fold_metrics, axis=0)
        results.append({
            "SMOTE Technique": smote_name,
            "Model": model_name,
            "F1-Score": avg_f1,
            "AUC-ROC": avg_auc,
            "AUC-PR": avg_auc_pr
        })

# Tabela de resultados
results_df = pd.DataFrame(results)
print("\nResultados Comparativos:")
print(results_df)

# Salvar os resultados
try:
    results_df.to_csv("comparacao_smote_resultados.csv", index=False)
    print("Resultados salvos em 'comparacao_smote_resultados.csv'")
except Exception as e:
    print(f"Erro ao salvar resultados: {e}")



Aplicando SMOTE com Decision Tree


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Aplicando SMOTE com Random Forest


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Aplicando SMOTE com SVM


...


Aplicando SMOTE com XGBoost


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Aplicando SMOTE com Gradient Boosting


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Resultados Comparativos:
  SMOTE Technique              Model  F1-Score   AUC-ROC    AUC-PR
0           SMOTE      Decision Tree  0.494546  0.858451  0.517795
1           SMOTE      Random Forest  0.500735  0.883327  0.617526
2           SMOTE                SVM  0.228289  0.738636  0.431222
3           SMOTE        Naive Bayes  0.385142  0.775643  0.379646
4           SMOTE                KNN  0.437395  0.821090  0.481009
5           SMOTE            XGBoost  0.524243  0.887650  0.625622
6           SMOTE  Gradient Boosting  0.522651  0.885747  0.620403
Resultados salvos em 'comparacao_smote_resultados.csv'
